In [2]:
# Import packages

import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import pandas as pd
from scipy.signal import medfilt
import functools
from scipy.optimize import minimize, bisect
import os
import seaborn as sns

In [3]:
pwd

u'/Users/indre/Desktop/Thesis Data Processing'

In [4]:
cd '/Users/indre/Python/dp_python-master'

/Users/indre/Python/dp_python-master


In [5]:
from dpcore import dp

In [6]:
%matplotlib qt

In [7]:
!pip install PyShp

In [8]:
# Define functions
# From Sylvester et al. (2019)

def compute_derivatives(x,y):
    """function for computing first derivatives of a curve (centerline)
    x,y are cartesian coodinates of the curve
    outputs:
    dx - first derivative of x coordinate
    dy - first derivative of y coordinate
    ds - distances between consecutive points along the curve
    s - cumulative distance along the curve"""
    dx = np.gradient(x) # first derivatives
    dy = np.gradient(y)   
    ds = np.sqrt(dx**2+dy**2)
    s = np.cumsum(ds)
    return dx, dy, ds, s
    dx,dy,ds,s

def compute_curvature(x,y):
    """function for computing first derivatives and curvature of a curve (centerline)
    x,y are cartesian coodinates of the curve
    outputs:
    dx - first derivative of x coordinate
    dy - first derivative of y coordinate
    ds - distances between consecutive points along the curve
    s - cumulative distance along the curve
    curvature - curvature of the curve (in 1/units of x and y)"""
    dx = np.gradient(x) # first derivatives
    dy = np.gradient(y)      
    ds = np.sqrt(dx**2+dy**2)
    ddx = np.gradient(dx) # second derivatives 
    ddy = np.gradient(dy) 
    curvature = (dx*ddy - dy*ddx) / ((dx**2 + dy**2)**1.5)
    s = np.cumsum(ds)
    return dx, dy, ds, s, curvature
    
def resample_curve(x,y,deltas,sf):
    dx,dy,ds,s = compute_derivatives(x,y)
    tck, u = scipy.interpolate.splprep([x,y],s=sf) 
    unew = np.linspace(0,1,1+s[-1]/deltas) # vector for resampling
    out = scipy.interpolate.splev(unew,tck) # resampling
    return out[0], out[1]

def correlate_clines(x1,x2,y1,y2):
    # use dynamic time warping to correlate centerlines
    c = len(x1)
    r = len(x2)
    sm = np.zeros((r,c))
    for i in range(0,r):
        sm[i,:] = ((x1-x2[i])**2 + (y1-y2[i])**2)**0.5
    p,q,C,phi = dp(sm,penalty=0.0,gutter=0.0)
    return p,q,sm

In [9]:
pwd

u'/Users/indre/Python/dp_python-master'

In [10]:
cd '/Users/indre/Desktop/Thesis Data Processing'

/Users/indre/Desktop/Thesis Data Processing


In [11]:
#  Import CVS Files 
#  In meters. Change columns to 2, 3 for feet.

df = pd.read_csv('/Users/indre/Desktop/Thesis Data Processing/upper_bank_v3.csv',usecols=[2,3])
df.columns = ['x', 'y']
rbx = np.array(df['x'])
rby = np.array(df['y'])

df = pd.read_csv('/Users/indre/Desktop/Thesis Data Processing/lower_bank_v3.csv',usecols=[2,3])
df.columns = ['x', 'y']
lbx = np.array(df['x'])
lby = np.array(df['y'])

df = pd.read_csv('/Users/indre/Desktop/Thesis Data Processing/centerline_v3.csv',usecols=[2,3])
df.columns = ['x', 'y']
x = np.array(df['x'])
y = np.array(df['y'])

df = pd.read_csv('/Users/indre/Desktop/Thesis Data Processing/thalweg_lb_v2.csv', usecols=[2,3]) 
df.columns = ['x', 'y']
df[:5]

tlbx = np.array(df['x'])
tlby = np.array(df['y'])

df = pd.read_csv('/Users/indre/Desktop/Thesis Data Processing/thalweg_rb_v2.csv',usecols=[2,3])
df.columns = ['x', 'y']

trbx = np.array(df['x'])
trby  = np.array(df['y'])

df = pd.read_csv('/Users/indre/Desktop/Thesis Data Processing/broad_thalweg_lower_v8.csv', usecols=[2,3])
df.columns = ['x', 'y']

btlbx = np.array(df['x'])
btlby = np.array(df['y'])

df = pd.read_csv('/Users/indre/Desktop/Thesis Data Processing/broad_thalweg_rb.csv',usecols=[2,3])
df.columns = ['x', 'y']

btrbx = np.array(df['x'])
btrby = np.array(df['y'])

In [12]:
# Resample centerlines

import scipy.interpolate

deltas = 25  # sampling distance, previously and in half width analysis is 50m
sf = 20000 # smoothing factor changed from 200000 and then half that and then 20000

btlbx, btlby = resample_curve(btlbx,btlby,deltas,sf)
btrbx, btrby = resample_curve(btrbx,btrby,deltas,sf)

x, y = resample_curve(x,y,deltas,sf)
lbx, lby = resample_curve(lbx,lby,deltas,sf)
rbx, rby = resample_curve(rbx,rby,deltas,sf)

/Users/indre/anaconda3/envs/DTW/lib/python2.7/site-packages/ipykernel_launcher.py:37: DeprecationWarning: object of type <type 'numpy.float64'> cannot be safely interpreted as an integer.


In [12]:
# Plot widths

plt.figure(figsize=(8,6))

#  centerline
plt.plot(x/1000,y/1000,'b')

# levee crest
plt.plot(lbx/1000,lby/1000,'g')
plt.plot(rbx/1000,rby/1000,'g')

# thalweg
plt.plot(btlbx/1000,btlby/1000,'y')
plt.plot(btrbx/1000,btrby/1000,'y')
plt.xlim(575.500,579.000)
plt.ylim(3041.000,3045.000)

plt.ylabel('distance from south to north (km)', fontsize =12); 
plt.xlabel('distance from east to west (km)',fontsize =12)
plt.ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
plt.ticklabel_format(axis='y', style='sci', scilimits=(-2,3))

In [16]:
# Compute curvature

dx, dy, ds, s, curv = compute_curvature(x,y)
from scipy.signal import savgol_filter
curv = savgol_filter(curv,51,3)

In [15]:
curv_s = savgol_filter(curv,51,3)

In [ ]:
# Now, resample and smooth left and right bank width, and thalweg width coordinates
# Compute widths by correlating to centerline

In [16]:
rbx, rby = resample_curve(rbx,rby,deltas,sf)

/Users/indre/anaconda3/envs/DTW/lib/python2.7/site-packages/ipykernel_launcher.py:37: DeprecationWarning: object of type <type 'numpy.float64'> cannot be safely interpreted as an integer.


In [17]:
lbx, lby = resample_curve(lbx,lby,deltas,sf)

/Users/indre/anaconda3/envs/DTW/lib/python2.7/site-packages/ipykernel_launcher.py:37: DeprecationWarning: object of type <type 'numpy.float64'> cannot be safely interpreted as an integer.


In [19]:
btlbx, btlby = resample_curve(btlbx,btlby,deltas,sf)
btrbx, btrby = resample_curve(btrbx,btrby,deltas,sf)

/Users/indre/anaconda3/envs/DTW/lib/python2.7/site-packages/ipykernel_launcher.py:37: DeprecationWarning: object of type <type 'numpy.float64'> cannot be safely interpreted as an integer.


In [20]:
pr,qr,smr = correlate_clines(x,rbx,y,rby) 

In [21]:
pl,ql,sml = correlate_clines(x,lbx,y,lby) 

In [22]:
btpr,btqr,btsmr = correlate_clines(x,btrbx,y,btrby) 

In [23]:
btpl,btql,btsml = correlate_clines(x,btlbx,y,btlby) 

In [24]:
qnr = np.delete(np.array(qr),np.where(np.diff(qr)==0)[0]+1)
pnr = np.delete(np.array(pr),np.where(np.diff(qr)==0)[0]+1)

In [25]:
qnl = np.delete(np.array(ql),np.where(np.diff(ql)==0)[0]+1)
pnl = np.delete(np.array(pl),np.where(np.diff(ql)==0)[0]+1)

In [26]:
btqnr = np.delete(np.array(btqr),np.where(np.diff(btqr)==0)[0]+1)
btpnr = np.delete(np.array(btpr),np.where(np.diff(btqr)==0)[0]+1)

btqnl = np.delete(np.array(btql),np.where(np.diff(btql)==0)[0]+1)
btpnl = np.delete(np.array(btpl),np.where(np.diff(btql)==0)[0]+1)

In [27]:
rbw = smr[pnr,qnr]
lbw = sml[pnl,qnl]  

In [28]:
btrbw = btsmr[btpnr,btqnr]
btlbw = btsml[btpnl,btqnl]

In [37]:
# Plot left and right bank width, first panel

plt.figure(figsize=(15,6))

#RBW Axis
plt.plot(s/1000,rbw)
plt.ylabel('width (m)', fontsize=14)
plt.xlabel('distance along channel axis (km)', fontsize=14) 

#LBW Axis
plt.plot(s/1000,lbw)

#Average half width Axis
plt.plot(s/1000,0.5*lbw+0.5*rbw)

#Legend
plt.legend(['right bank width','left bank width','mean half width']);
plt.title('distance between banks and centerline', fontsize =16);

In [71]:
# Plot left and right bank width, second panel

plt.figure(figsize=(15,6))

#RBW Axis
plt.plot(s/1000,rbw)
plt.ylabel('width (m)', fontsize=14)
plt.xlabel('distance along channel axis (km)', fontsize=14) 
plt.xlim(0,226.39785743974724) 

#LBW Axis
plt.plot(s/1000,lbw) 

#Average half width Axis
plt.plot(s/1000,0.5*lbw+0.5*rbw)

#legend
plt.legend(['right bank width','left bank width', 'mean half width']);

In [376]:
# Plot left and right bank width, third panel

plt.figure(figsize=(15,6))

#RBW Axis
plt.plot(s/1000,rbw)
plt.ylabel('width (m)', fontsize=14)
plt.xlabel('distance along channel axis (km)', fontsize=14) 
plt.xlim(226.39785743974724*2,max(s/1000)) 

#LBW Axis
plt.plot(s/1000,lbw) 

#Average half width Axis
plt.plot(s/1000,0.5*lbw+0.5*rbw)

#legend
plt.legend(['right bank width','left bank width', 'mean half width']);

In [72]:
# Compare HW and Curvature

fig = plt.figure(figsize=(14,4))
ax1 = fig.add_subplot(111)

#Curvature Axis
ax1.plot(s/1000,curv_s,'r'); 

#Label curv_s=0
ax1.plot([0, max(s/1000)],[0,0],'k--')

ax1.set_ylabel('curvature (1/m)', fontsize=14)
ax1.set_xlabel('distance along channel axis (km)', fontsize=14) #CHECK UNITS

# Width Axis
ax2 = ax1.twinx()
ax2.plot(s/1000,rbw)
ax2.set_ylabel('width (m)', fontsize=14)

#legend
ax1.legend(['curvature'],loc='upper left',);
ax2.legend(['right bank width'],loc='upper right');

In [121]:
# Correlation plot
# Only 5% of data points are plotted as black dots. 

plt.figure(figsize=(8,8))
sns.kdeplot(curv[:23], rbw[-23:],
           n_levels=20,shade=True,cmap='Blues', shade_lowest=False)
plt.scatter(curv[:23][::20],rbw[-23:][::20],c='k',s=10)
plt.xlabel('curvature (1/m)', fontsize=14)
plt.ylabel('right bank width, with phase lag (m)', fontsize=14)

Text(0,0.5,'right bank width, with phase lag (m)')

In [32]:
# 5. Import bends* from CSV 

# * To edit, see "bend_picker" in /Desktop/Thesis Data Processing and change file description below to "..._NEW"

df = pd.read_csv('/Users/indre/Desktop/Thesis Data Processing/Joshua_bathy_lzc_lzm.csv', usecols=[0,1,2,3,4])
df.columns = ['bend','index_inflection_point','index_zero_migration','s_coordinate_index_ip','s_coordinate_index_zm']
                 
BEND = np.array(df['bend'])
LZC = np.array(df['index_inflection_point'])
LZM = np.array(df['index_zero_migration'])
LZC_s = np.array(df['s_coordinate_index_ip'])
LZM_s = np.array(df['s_coordinate_index_zm'])

LZC=LZC.astype(int)
LZM=LZM.astype(int)


In [33]:
# Rough estimate of lag

print np.median(LZM-LZC)
print np.median(LZM-LZC)*deltas

300.0

In [129]:
# Iterate estimate for maximum correlation to find lag

# Without lag
from scipy.stats import pearsonr
corr, _ = pearsonr(curv, rbw)
print('Pearsons correlation: %.3f' % corr)

# With lag!
from scipy.stats import pearsonr
# calculate Pearson's correlation
corr, _ = pearsonr(curv[:-10], rbw[10:])
print('Pearsons correlation: %.3f' % corr)


12

In [136]:
# 6. Map bends, cropped 
# colors: hz1 - 'b', hz 2 - 'g' , hz3 - 'c', hz 4 - 'y'

plt.figure(figsize=(60,60))
           
plt.plot(x, y)
plt.axis('equal')
plt.scatter(x[LZC], y[LZC], c='r')
plt.scatter(x[LZM], y[LZM], c='b')  

for i, txt in enumerate(BEND):
    plt.annotate(txt, (x[LZC][i], y[LZC][i]), color='r',  size='small')
for i, txt in enumerate(BEND):
    plt.annotate(txt, (x[LZM][i], y[LZM][i]), color='b', size='small')
    
LZM=LZM.astype(int)
LZC=LZC.astype(int)
plt.ylabel('distance from south to north (m)', fontsize =12); 
plt.xlabel('distance from east to west (m)',fontsize =12)
plt.ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
plt.ticklabel_format(axis='y', style='sci', scilimits=(-2,2))

In [157]:
# Make zero crossings for half width

rbw = rbw - np.median(rbw) 


In [158]:
# 7. Plot data by bends

W = 174

fig, ax1 = plt.subplots(figsize=(16,4))

y1g = 0.5 # top for curv
y2g = 0 # zero for curv in migr
y3g = -0.8333 # zero for migr in curv
y4g = -1.3 # bottom for curv

for i in range(1,len(LZC)-1,2):
    xcoords = [LZC_s[i],LZC_s[i+1],LZC_s[i+1],LZM_s[i+1],LZM_s[i+1],LZM_s[i],LZM_s[i],LZC_s[i]]
    ycoords = [y1g,y1g,y2g,y3g,y4g,y4g,y3g,y2g]
    ax1.fill(xcoords,ycoords,color=[0.85,0.85,0.85],zorder=0) 
        
offset = 10
deltas = 25.0

ax1.fill_between(s, 0, curv*W) 
ax2.fill_between(s, 0, rbw, facecolor='green')

ax1.plot([0,max(s)],[0,0],'k--')
ax2.plot([0,max(s)],[0,0],'k--')

ax1.set_ylim(y4g,y1g)
ax2.set_ylim(-350,1000)
ax1.set_xlim(0,s[-1])
   
ax1.set_ylabel('W/r (m)')
ax2.set_ylabel('migration distance (m)')
ax1.set_xlabel("distance along channel axis (m)")

plt.tight_layout()

In [142]:
# 8. Correlation plots

plt.figure(figsize=(8,8))
sns.kdeplot(curv[:-10], rbw[10:],
           n_levels=20,shade=True,cmap='Blues', shade_lowest=False)
plt.scatter(curv[:-10][::20],rbw[10:][::20],c='k',s=10)
plt.xlabel('curvature (1/m)', fontsize=14)
plt.ylabel('right bank width, with phase lag (m)', fontsize=14)

Text(0,0.5,'right bank width, with phase lag (m)')